In [1]:
%run ../__init__.py

## Data preprocessing with Tfidf

In [2]:
budget_train = pd.read_csv('../data/train_cleaned.csv', index_col=0)

In [3]:
budget_train.head()

,Function,Use,Sharing,Reporting,Student_Type,Position_Type,Object_Type,Pre_K,Operating_Status,Object_Description,...,Location_Description,FTE,Function_Description,Facility_or_Department,Position_Extra,Total,Program_Description,Fund_Description,Text_1,combined_text
0,Function__Teacher Compensation,Use__Instruction,Sharing__School Reported,Reporting__School,Student_Type__NO_LABEL,Position_Type__Teacher,Object_Type__NO_LABEL,Pre_K__NO_LABEL,Operating_Status__PreK-12 Operating,,...,,1.000000,,,kindergarten,50471.810,kindergarten,general fund,,teacher elementary kindergar...
1,Function__NO_LABEL,Use__NO_LABEL,Sharing__NO_LABEL,Reporting__NO_LABEL,Student_Type__NO_LABEL,Position_Type__NO_LABEL,Object_Type__NO_LABEL,Pre_K__NO_LABEL,Operating_Status__Non-Operating,contractor service,...,,0.420663,rgn gob,,undesignated,3477.860,building improvement service,,building improvement service,contractor service bond expenditure building f...
2,Function__Teacher Compensation,Use__Instruction,Sharing__School Reported,Reporting__School,Student_Type__Unspecified,Position_Type__Teacher,Object_Type__Base Salary/Compensation,Pre_K__Non PreK,Operating_Status__PreK-12 Operating,personal service teacher,...,,1.000000,,,teacher,62237.130,instruction regular,general purpose school,,personal service teacher tcher 2nd grade ...
3,Function__Substitute Compensation,Use__Instruction,Sharing__School Reported,Reporting__School,Student_Type__Unspecified,Position_Type__Substitute,Object_Type__Benefits,Pre_K__NO_LABEL,Operating_Status__PreK-12 Operating,employee benefit,...,,0.420663,unalloc budget school,,professional instructional,22.300,general middle junior high sch,,regular instruction,employee benefit teacher sub general fund teac...
4,Function__Substitute Compensation,Use__Instruction,Sharing__School Reported,Reporting__School,Student_Type__Unspecified,Position_Type__Teacher,Object_Type__Substitute Compensation,Pre_K__NO_LABEL,Operating_Status__PreK-12 Operating,teacher coverage teacher,...,,0.420663,non project,,professional instructional,54.166,general high school education,,regular instruction,teacher coverage teacher teacher sub general f...


In [9]:
budget_train.isnull().sum()

Function                  0
Use                       0
Sharing                   0
Reporting                 0
Student_Type              0
Position_Type             0
Object_Type               0
Pre_K                     0
Operating_Status          0
Object_Description        0
Text_2                    0
SubFund_Description       0
Job_Title_Description     0
Text_3                    0
Text_4                    0
Sub_Object_Description    0
Location_Description      0
FTE                       0
Function_Description      0
Facility_or_Department    0
Position_Extra            0
Total                     0
Program_Description       0
Fund_Description          0
Text_1                    0
combined_text             0
dtype: int64

In [5]:
budget_test = pd.read_csv('../data/test_clean.csv', index_col=0)

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
budget_test.isnull().sum()

ID                        0
Object_Description        0
Program_Description       0
SubFund_Description       0
Job_Title_Description     0
Facility_or_Department    0
Sub_Object_Description    0
Location_Description      0
FTE                       0
Function_Description      0
Position_Extra            0
Text_4                    0
Total                     0
Text_2                    0
Text_3                    0
Fund_Description          0
Text_1                    0
dtype: int64

In [7]:
for col in budget_test[word_predictors]:
    budget_test[col] = budget_test[word_predictors].loc[:,[col]].fillna(' ')

In [8]:
for col in budget_train[word_predictors]:
    budget_train[col] = budget_train[word_predictors].loc[:,[col]].fillna(' ')

In [16]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import Imputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression

In [41]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler

In [12]:
label = LabelEncoder()

In [58]:
class ItemSelector(BaseEstimator, TransformerMixin):
    """For data grouped by feature, select subset of data at a provided key.

    The data is expected to be stored in a 2D data structure, where the first
    index is over features and the second is over samples.  i.e.

    >> len(data[key]) == n_samples

    Please note that this is the opposite convention to scikit-learn feature
    matrixes (where the first index corresponds to sample).

    ItemSelector only requires that the collection implement getitem
    (data[key]).  Examples include: a dict of lists, 2D numpy array, Pandas
    DataFrame, numpy record array, etc.

    >> data = {'a': [1, 5, 2, 5, 2, 8],
               'b': [9, 4, 1, 4, 1, 3]}
    >> ds = ItemSelector(key='a')
    >> data['a'] == ds.transform(data)

    Parameters
     ----------
    key : hashable, required
        The key corresponding to the desired value in a mappable.
    """
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]

    

In [59]:
# 400,000+ examples, 14 features, return labels in {0, 1, 2}
X = budget_train[word_predictors] 
y = label.fit_transform(budget_train['Operating_Status'])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = .2, random_state=42)
                                                

In [ ]:
# feature union transformer list 
transformer_list = [
    ('Object_Description', Pipeline([
                ('selector', ItemSelector(key='Object_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=10)),
            ])),
    ('Text_2', Pipeline([
                ('selector', ItemSelector(key='Text_2')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=10)),
            ])),
    ('SubFund_Description', Pipeline([
                ('selector', ItemSelector(key='SubFund_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=10)),
            ])),
    ('Job_Title_Description', Pipeline([
                ('selector', ItemSelector(key='Job_Title_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=10)),
            ])),
    ('Text_3', Pipeline([
                ('selector', ItemSelector(key='Text_3')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=10)),
            ])),
    ('Text_4', Pipeline([
                ('selector', ItemSelector(key='Text_4')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=10)),
            ])),
    ('Sub_Object_Description', Pipeline([
                ('selector', ItemSelector(key='Sub_Object_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=10)),
            ])),
    ('Location_Description', Pipeline([
                ('selector', ItemSelector(key='Location_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=10)),
            ])),
    ('Function_Description', Pipeline([
                ('selector', ItemSelector(key='Function_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=10)),
            ])),
    ('Facility_or_Department', Pipeline([
                ('selector', ItemSelector(key='Facility_or_Department')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=10)),
            ])),
    ('Position_Extra', Pipeline([
                ('selector', ItemSelector(key='Position_Extra')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=10)),
            ])),
    ('Program_Description', Pipeline([
                ('selector', ItemSelector(key='Program_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=10)),
            ])),
    ('Fund_Description', Pipeline([
                ('selector', ItemSelector(key='Fund_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=10)),
            ])),
    ('Text_1', Pipeline([
                ('selector', ItemSelector(key='Text_1')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=10)),
            ])),
    ('FTE', Pipeline([
                ('selector', ItemSelector(key='FTE')),
                ('scaler', StandardScaler()),
            ])),
#     ('Total', Pipeline([
#                 ('selector', ItemSelector(key='Total')),
#                 ('scaler', StandardScaler()),
#             ])),
   
]

# create pipeline
pipeline = Pipeline([
    ("union", FeatureUnion(transformer_list=transformer_list)),
    ("clf", LogisticRegression()),
])

In [44]:
scaler = StandardScaler()

In [87]:
type(budget_train['FTE'])

pandas.core.series.Series

In [88]:
fte = budget_train['FTE'].values.reshape(-1,1)

In [90]:
type(fte)

numpy.ndarray

In [107]:
fte_transformed = scaler.fit_transform(budget_train['FTE'])

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


In [96]:
len(budget_train[num_predictors])

400277

In [94]:
fte_transformed

array([[  1.00000000e+00,   5.04718100e+04],
       [  4.20663015e-01,   3.47786000e+03],
       [  1.00000000e+00,   6.22371300e+04],
       ..., 
       [  1.00000000e+00,   4.90229000e+03],
       [  4.20663015e-01,   4.02029000e+03],
       [  4.20663015e-01,   4.65300000e+01]])

In [93]:
type(fte_transformed)

numpy.ndarray

In [71]:
X_test = budget_test

In [109]:
Ops_model = pipeline.fit(X_train, y_train)

KeyError: 'FTE'

In [126]:
Ops_model.score(X_train, y_train)

0.98528203959140714

In [128]:
predict_proba = Ops_model.predict_proba(X_val)

In [129]:
log_loss(y_val, predict_proba)

0.047490405186274731

In [130]:
label.classes_

array(['Operating_Status__Non-Operating',
       'Operating_Status__Operating, Not PreK-12',
       'Operating_Status__PreK-12 Operating'], dtype=object)

In [127]:
from sklearn.externals import joblib

In [133]:
joblib.dump(Ops_model, 'Ops_status_model.pkl')

['Ops_status_model.pkl']